In [27]:
!pip install qdrant-client

In [28]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/alfredodeza/learn-retrieval-augmented-generation/refs/heads/main/top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
#data = df.to_dict('records')
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


In [29]:
df
new_record = {'name': 'Intipalka', 'region': 'Ica', 'variety': 'Red wine', 'rating': 56, 'notes': 'año nuevo'}
#df.loc[len(df)] = new_record
df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
data = df.to_dict('records')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     1348 non-null   object 
 1   region   1348 non-null   object 
 2   variety  1348 non-null   object 
 3   rating   1348 non-null   float64
 4   notes    1348 non-null   object 
dtypes: float64(1), object(4)
memory usage: 52.8+ KB


In [30]:
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1343,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1344,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1345,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,
1346,Lewis Cellars Reserve Cabernet Sauvignon (scuf...,"Napa Valley, California",Red Wine,96.0,"The headline reads: ""2012 Reserve Cabernet - R..."


In [31]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [32]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [33]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [34]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-34-5de2cf0f8439>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [35]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [36]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("Algo para año nuevo").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Intipalka', 'region': 'Ica', 'variety': 'Red wine', 'rating': 56.0, 'notes': 'año nuevo'} score: 0.842394915817236
{'name': 'Artadi Pagos Viejos 1995', 'region': 'Rioja, Spain', 'variety': 'Red Wine', 'rating': 96.0, 'notes': "Many say Artadi is one of the most sought-after estates in Rioja and Pagos Viejos one of the most sought-after wines in the world. Decades of intense labor in the vineyards have given famed winemaker and owner Juan Carlos Lopez de Lacalle the experience and passion for making wines of true excellence. Containing fine aromas and an elegant mouth feel that is fruit-forward on the mid-palate, and supported by round tannins, Artadi's Pagos Viejos is the ideal representation of what an artisanal wine should taste like when carefully made from the old-vine Tempranillo that is grown in Rioja Alavesa. An experience to be enjoyed on special occasions, this wine is dark cherry red in color and is both elegant and aromatic. Containing intriguing aromas of dark cho

<ipython-input-36-1f7fdb382c7c>:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(
